In [ ]:
# Important note: This workflow shown on this notebook is only an example
# on how to work with Data API and Orders API. It is NOT a definite or
# the only way to streamline Data/Orders API process. You're free to modify and 
# adjust to your workflow. Planet is NOT responsible or liable in case of
# any loss occured when you're using this notebook. 

In [ ]:
# Import all required modules
from __future__ import print_function
import requests
import time
import os
import folium
import json
from requests.auth import HTTPBasicAuth
import geopandas
from shapely.geometry import shape

# Insert your api key here
PLANET_API_KEY = "planet_api_key"

# 1. Find imagery using Data API

In [ ]:
# Get your AOI files
aoi_fpath = r"my_aoi.geojson" 
with open(aoi_fpath) as f:
    geom = json.load(f)

with open(aoi_fpath) as AOI_geom:
    data = json.load(AOI_geom)['features']
    my_aoi = data[0]['geometry']

In [ ]:
# Double check the AOI
display_map = folium.Map(tiles='OpenStreetMap', zoom_start=9)
folium.GeoJson(my_aoi, name='geojson').add_to(display_map)
display_map

In [ ]:
# get images that overlap with your AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": my_aoi
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2022-07-01T00:00:00.000Z",
    "lte": "2022-08-02T00:00:00.000Z"
  }
}

# only get images which have <15% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_percent",
  "config": {
    "lte": 15
  }
}

# only get images captured with PSB.SD (SuperDoves) - other instrument PS2 (Dove Classic), PS2.SD (Dove-R) 
instrument_filter = {
   "type":"StringInFilter",
   "field_name":"instrument",
   "config":["PSB.SD"]
}

# only get standard quality images
quality_filter = {
   "type":"StringInFilter",
   "field_name":"quality_category",
   "config":["standard"]
}

# combine our geo, date, cloud filters - other filters can be reffered in https://developers.planet.com/docs/apis/data/searches-filtering/
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter, instrument_filter, quality_filter]
}

In [ ]:
# Defined item-type - other item type can be referred in https://developers.planet.com/docs/apis/data/items-assets/
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request to Data API/quick-search endpoint
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',auth=HTTPBasicAuth(PLANET_API_KEY,''),
    json=search_request)

result = search_result.json()

# List all the image ids
image_ids = [feature['id'] for feature in search_result.json()['features']] 
print(image_ids)

# 2. Visualize footprint results (Optional)

In [ ]:
# Get image footprints from results
item_footprint = geopandas.GeoDataFrame.from_features(result['features'])
item_footprint.crs = 4326 
item_footprint_geom = item_footprint.to_json()

In [ ]:
# Double check
display_map = folium.Map(tiles='OpenStreetMap', zoom_start=9)
folium.GeoJson(item_footprint_geom, name='footprint').add_to(display_map)
folium.GeoJson(my_aoi, name='aoi').add_to(display_map)
folium.LayerControl().add_to(display_map)
display_map

# 3. Downloading selected images via Orders API

In [ ]:
# Prepare the payload - additional raster tool can be referred in https://developers.planet.com/docs/orders/tools-reference/ 
# Item-type and product bundle reference https://developers.planet.com/docs/orders/product-bundles-reference/ 
payload = {
  "name": "my imagery order",
  "products":[
    {
      "item_ids": image_ids,
     "item_type": "PSScene",
    "product_bundle": "analytic_udm2"
    }
  ],
  "tools": [
    {
      "clip": {
        "aoi": my_aoi
      }
    }
  ]
}

print(payload)

In [ ]:
headers = {
    'Content-Type': 'application/json'
    }
order = \
  requests.post('https://api.planet.com/compute/ops/orders/v2/', headers=headers, data=json.dumps(payload), auth=HTTPBasicAuth(PLANET_API_KEY,''))

response = order.json()
print(response)

In [ ]:
# Get order status
state = "running"
while state == "running":
    return_data = requests.get(order.json()['_links']['_self'],auth=(PLANET_API_KEY, ''))
    state = return_data.json()['state']
    print("status: ",state)
    if state == 'running':
        time.sleep(25)

In [ ]:
# Start downloading

def download_file(url,name):
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True,auth=(PLANET_API_KEY, ''))
    with open(name, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        i = 0
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                i += 1024
                print("Downloading (%)... {}".format(round(i/float(total_length)*100,4)),end='\r')
        print("Downloading (%)... 100.0000",end='\r')
    return 0

In [ ]:
download_dir = 'C:\\Users\\emir\\Documents\\Project\\Customer Onboarding\\CSE-736\\'
count = 0
for val in return_data.json()['_links']['results']:
    if "AnalyticMS_clip" in str(val['name']):
        count += 1
#         print (download_dir+val['name'].split("/")[-1])
        print ('\nFile {}/{}'.format(count, len(image_ids)))
        download_file(val['location'], download_dir+val['name'].split("/")[-1])